# zhsegment

## Run the default solution on dev

In [ ]:
Pw = Pdist(data=datafile("data/count_1w.txt"))
segmenter = Segment(Pw) # note that the default solution for this homework ignores the unigram counts
output_full = []
with open("data/input/dev.txt") as f:
    for line in f:
        output = " ".join(segmenter.segment(line.strip()))
        output_full.append(output)
print("\n".join(output_full[:3])) # print out the first three lines of output as a sanity check

## Evaluate the default output

In [ ]:
from zhsegment_check import fscore
with open('data/reference/dev.out', 'r') as refh:
    ref_data = [str(x).strip() for x in refh.read().splitlines()]
    tally = fscore(ref_data, output_full)
    print("score: {:.2f}".format(tally), file=sys.stderr)


## Documentation

We developed an iterative segmenter.

##### Class Entry: 
Each entry in the chart has four components: Entry(word, start-position, log-probability, back-pointer), the back-pointer in each entry links it to a previous entry that it extends.

In [6]:
class Entry():
    def __init__(self, word, end, logprob, backptr):
        self.word = word
        self.end = end
        self.logprob = logprob
        self.backptr = backptr
    def __lt__(self, other):
        if self.logprob>other.logprob:
            return True
        else:
            return False

##### Initialize the heap

In [9]:
# find the matched word start with char
def inputMatch(self, position, text, prevEntry):
    entryList = []
    char = text[position]
    #for all the words within the max length
    for i in range( min((self.Pw).maxLength, len(text)-position) ):
        word = text[position:position+i+1]
        if word in (self.Pw).keys():
            if prevEntry is None:
                logprob = log10(self.Pw(word))
            else:
                logprob = log10(self.cPw(word, prevEntry.word))
            end = position + i+1
            entryList.append(Entry(word, end, logprob, None))
    # return the words with coming 2 chars if no matched
    if len(entryList) == 0:
        logprob = log10(self.Pw(char))
        end = position + 1
        entryList.append(Entry(char, end, logprob, None))
        # entryList.append(Entry(char, end, logprob, None))
        # entryList.append(Entry(char, end, logprob, None))
    return entryList

##### Iteratively fill in chart[i] for all i

In [12]:
chart = [None]*len(text)
while len(entryHeap) != 0:
    entry = heapq.heappop(entryHeap)
    endindex = entry.end - 1
    if chart[endindex] is not None:
        preventry = chart[endindex]
        if entry.logprob > preventry.logprob:
            chart[endindex] = entry
        else:
            continue
    else:
        chart[endindex] = entry
        #if the text does not finish
        if (endindex+1) < len(text):
            entryList = self.inputMatch(endindex+1, text, chart[endindex])
            for newEntry in entryList:
                newStandardEntry = Entry(newEntry.word, newEntry.end, newEntry.logprob+entry.logprob, endindex)
                if any(newStandardEntry == existEntry for existEntry in entryHeap) is not True:
                    heapq.heappush(entryHeap, newStandardEntry)

##### Get the best segmentation

In [ ]:
finalEntry = chart[-1]
currentEntry = finalEntry
segmentation = []
segmentation.append(currentEntry.word)
while currentEntry.backptr is not None:
    currentEntry = chart[currentEntry.backptr]
    segmentation.append(currentEntry.word)
segmentation.reverse()

##### Bigram model and Smoothing
We tried Laplace Add-1 smoothing as well as JM smoothing for a better F-score.

In [ ]:
def cPw(self, word, prev):
        "Conditional probability of word, given previous word."
        #Laplace Add-1 smoothing
        # if prev + ' ' + word in self.P2w:
        #     if prev in self.Pw:
        #         return (1 + self.P2w[prev + ' ' + word])/(len(self.Pw)+float(self.Pw[prev]))
        #     else:
        #         return (1 + self.P2w[prev + ' ' + word])/(len(self.Pw))
        # else:
        #     if prev in self.Pw:
        #         return 1 / (len(self.Pw)+float(self.Pw[prev]))
        #     else:
        #         return 1 / len(self.Pw)
        # Jelinek-Mercer smoothing
        bigramProb = 0
        unigramProb = 0
        if prev + ' ' + word in self.P2w:
            bigramProb = self.P2w[prev + ' ' + word] / float(self.Pw[prev])
        if word in self.Pw:
            unigramProb = self.Pw(word)
        bigram_lambda = 0.5
        unigram_lambda = 0.3
        return bigram_lambda*bigramProb + unigram_lambda*unigramProb + (1-bigram_lambda-unigram_lambda)*(1/float(len(self.Pw)))